In [ ]:
# IO
out_fp = "output/"
out_fn = out_fp + "kernel_test"

In [ ]:
# MCMC settings
mvi = 1
mni = 1
n_gen = 1e4
sample_freq = floor(n_gen/1e2)

In [ ]:
# data dimensions
n_taxa = 10
n_sites = 100

In [ ]:
# create dummy taxa
for (i in 1:n_taxa) {
    taxa[i] = taxon("T"+i, 0.0)
}
taxa

In [ ]:
# create unit Yule tree
birth ~ dnExp(1)
phy ~ dnBDP(lambda=birth, mu=0., rootAge=1, taxa=taxa)

# tree moves
mv[mvi++] = mvNNI(phy, weight=n_taxa)
mv[mvi++] = mvFNPR(phy, weight=n_taxa/2)
mv[mvi++] = mvNodeTimeSlideUniform(phy, weight=n_taxa)

# print the tree's value
phy

In [ ]:
# base frequencies
pi ~ dnDirichlet(rep(1,4))
mv[mvi++] = mvSimplexElementScale(pi, alpha=10, weight=4)

# exchangeability rates
er ~ dnDirichlet(rep(1,6))
mv[mvi++] = mvSimplexElementScale(er, alpha=10, weight=6)

# GTR rate matrix
Q := fnGTR(exchangeRates=er,
           baseFrequencies=pi)
    
# print the matrix's value
Q

In [ ]:
# build phylogenetic CTMC
seq ~ dnPhyloCTMC(tree=phy,
                  Q=Q,
                  branchRates=1.,
                  nSites=n_sites,
                  type="DNA")

# treat the simulated data as the likelihood
seq.clamp(seq)

In [ ]:
# save some values
writeNexus(seq, filename=out_fn+".sim.nex")
writeNexus(phy, filename=out_fn+".sim.tre")

In [ ]:
# create monitors
mn[mni++] = mnScreen(pi, printgen=sample_freq)
mn[mni++] = mnModel(filename=out_fn+".model.log",
                    printgen=sample_freq)
mn[mni++] = mnFile(phy,
                   filename=out_fn+".tre",
                   printgen=sample_freq)

In [ ]:
# create MCMC
mdl = model(phy)
ch = mcmc(mdl,mv,mn)
ch.run(n_gen)